# Data Driven Pages

In [1]:
# This just configures the display of the jupyter notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import geopandas
import sqlalchemy

### Define Layer to Extract from ArcServer

In [3]:
#Set Project Database, Table and Columns
proj_database='MY_SQL_DB'
rpt_table='SEWER_MAIN_REFERENCE'
rpt_columns=['FACILITYID', 'GRID', 'LENGTH']

In [4]:
from pymssql import connect
import pandas as pd
from shapely.wkt import loads
from geopandas import GeoDataFrame
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
def rd_sql(server, database, table, col_names=None, where_col=None, where_val=None, geo_col=False, epsg=2193, export=False, path='save.csv'):
    """
    Imports data from MSSQL database, returns GeoDataFrame. Specific columns can be selected and specific queries within columns can be selected. Requires the pymssql package, which must be separately installed.
    Arguments:
    server -- The server name (str). e.g.: 'SQL2012PROD03'
    database -- The specific database within the server (str). e.g.: 'LowFlows'
    table -- The specific table within the database (str). e.g.: 'LowFlowSiteRestrictionDaily'
    col_names -- The column names that should be retrieved (list). e.g.: ['SiteID', 'BandNo', 'RecordNo']
    where_col -- The sql statement related to a specific column for selection (must be formated according to the example). e.g.: 'SnapshotType'
    where_val -- The WHERE query values for the where_col (list). e.g. ['value1', 'value2']
    geo_col -- Is there a geometry column in the table?
    epsg -- The coordinate system (int)
    export -- Should the data be exported
    path -- The path and csv name for the export if 'export' is True (str)
    """
    if col_names is None and where_col is None:
        stmt1 = 'SELECT * FROM ' + table
    elif where_col is None:
        stmt1 = 'SELECT ' + str(col_names).replace('\'', '"')[1:-1] + ' FROM ' + table
    else:
        stmt1 = 'SELECT ' + str(col_names).replace('\'', '"')[1:-1] + ' FROM ' + table + ' WHERE ' + str([where_col]).replace('\'', '"')[1:-1] + ' IN (' + str(where_val)[1:-1] + ')'
    conn = connect(server, database=database)
    df = pd.read_sql(stmt1, conn)

    ## Read in geometry if required
    if geo_col:
        geo_col_stmt = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME=" + "\'" + table + "\'" + " AND DATA_TYPE='geometry'"
        geo_col = str(pd.read_sql(geo_col_stmt, conn).iloc[0,0])
        if where_col is None:
            stmt2 = 'SELECT ' + geo_col + '.STGeometryN(1).ToString()' + ' FROM ' + table
        else:
            stmt2 = 'SELECT ' + geo_col + '.STGeometryN(1).ToString()' + ' FROM ' + table + ' WHERE ' + str([where_col]).replace('\'', '"')[1:-1] + ' IN (' + str(where_val)[1:-1] + ')'
        df2 = pd.read_sql(stmt2, conn)
        df2.columns = ['geometry']
        geometry = [loads(x) for x in df2.geometry]
        df = GeoDataFrame(df, geometry=geometry, crs={'init' :'epsg:' + str(epsg)})

    if export:
        df.to_csv(path, index=False)

    conn.close()
    return(df)

### Run Query and Load Dataframe

In [6]:
dataframe = rd_sql(server='MYSQLSERVER', database=proj_database, table=rpt_table, col_names=rpt_columns, geo_col=True, where_val='GDB_TO_DATE=9999-12-31 23:59:59.0000000', epsg=3857) 
dataframe['geometry'].shape

(2203,)

## Reproject to WGS84

In [7]:
from pyproj import CRS

In [8]:
print("Current Coordinate System: \n", dataframe.crs)
reprojected_df = dataframe.to_crs("EPSG:4326")
reprojected_df.crs

Current Coordinate System: 
 +init=epsg:3857 +type=crs


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
from shapely import wkt
data = reprojected_df.geometry.apply(lambda x: wkt.dumps(x))
df = pd.DataFrame(data) 
geometry=df['geometry'][4]

from shapely.wkt import loads
line_segment = loads(geometry)

### When creating labels and points in folium x,y coordinates need to be swapped

In [10]:
def swap_xy_coords(coords):
    x,y=coords
    x,y=y,x
    return float(x),float(y)
     
lateral=list(line_segment.coords)
lateral_swap=[]

for r in lateral:
    a=swap_xy_coords(r)
    lateral_swap.append(a)

lateral_swap_coords=[]
for row in lateral_swap:
    v = [row[0], row[1]]
    lateral_swap_coords.append(v)
    
print(lateral_swap_coords)

[[35.76655604022658, -78.61590974263426], [35.76580770229706, -78.6155728330802]]


### Constructing Initial Folium Map Layer Zoomed to Our Feature

In [11]:
# Import the folium library.
import folium
from folium import plugins

x, y = line_segment.coords.xy
# Defining Where to Center Map Zoom To
ave_lat = sum(x)/len(x)
ave_lon = sum(y)/len(y)
my_map = folium.Map(location=[ave_lon, ave_lat])

# Defining Dynamic Zoom Scale, by taking min/max bounds of line segment
sw = [min(y),min(x)]
ne = [max(y),max(x)]
my_map.fit_bounds([sw, ne])

#Display the map
my_map

### Adding in Markers and Polyline

In [12]:
#add a markers 
for each in lateral_swap_coords:  
    my_map.add_child(folium.CircleMarker(location=each,
    fill='true',
    radius = 8,
    popup= 'Manhole',
    fill_color='blue',
    color = 'clear',
    fill_opacity=1))

# add lines
my_PolyLine=folium.PolyLine(lateral_swap_coords, color="green", weight=3, opacity=1).add_to(my_map)
my_map.add_children(my_PolyLine)
my_map

### Authenticate Google Earth Engine
![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Google_Maps_icon.svg/240px-Google_Maps_icon.svg.png) 


In [ ]:
import ee
## Trigger the authentication flow. You only need to do this once
## Refer to the README.md for setting up Google Earth Engine access
ee.Authenticate()
# Initialize the library.
ee.Initialize()

### Test that Google API is working

In [14]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


# Interactive Map

The **folium library** can be used to display **ee.Image** objects on an interactive Leaflet map. Folium has no default method for handling tiles from Earth Engine, so one must be definedand added to the **folium.Map** module before use. The following cells provide an example of adding a method for handing Earth Engine tiles and using it to display an elevation model to a Leaflet map.

### Add Base Maps

In [15]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

## Defined add_ee_layer function()

In [16]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

## Display ee.Image tiles

In [17]:
# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
#my_map = folium.Map(location=[38.2527, -85.7585], zoom_start=11, height=500) # Louisville KY

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Display ee.Image
dataset = ee.Image('JRC/GSW1_1/GlobalSurfaceWater')
occurrence = dataset.select('occurrence');
occurrenceVis = {'min': 0.0, 'max': 100.0, 'palette': ['ffffff', 'ffbbbb', '0000ff']}

my_map.add_ee_layer(occurrence, occurrenceVis, 'JRC Surface Water')

# Display ee.Geometry
holePoly = ee.Geometry.Polygon(coords = [[[-35, -10], [-35, 10], [35, 10], [35, -10], [-35, -10]]],
                               proj= 'EPSG:4326',
                               geodesic = True,
                               maxError= 1.,
                               evenOdd = False)
my_map.add_ee_layer(holePoly, {}, 'Polygon')

# Display ee.FeatureCollection
fc = ee.FeatureCollection('TIGER/2018/States')
my_map.add_ee_layer(fc, {}, 'US States')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)
my_map.save('lat_line_example.html')

## Save As High Quality Image

In [18]:
# if you require an export other than html
import os
import time
from selenium import webdriver

delay=5
fn='lat_line_example.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
my_map.save(fn)

browser = webdriver.Firefox(executable_path='./geckodriver')
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('high_def_map.png')
browser.quit()

## Custom Markers

In [19]:
m = folium.Map(location=[ave_lon, ave_lat], zoom_start=22)

folium.Marker(
    location=[ave_lon, ave_lat], 
    icon=folium.Icon(color="red",icon="truck", prefix='fa')
).add_to(m)
# add lines
my_PolyLine=folium.PolyLine(lateral_swap_coords, color="green", weight=3, opacity=1, popup="Inspection: SEG1023").add_to(m)
m.add_children(my_PolyLine)


for each in lateral_swap_coords:  
    m.add_child(folium.CircleMarker(location=each,
    fill='true',
    radius = 8,
    popup= 'Manhole',
    fill_color='blue',
    color = 'clear',
    fill_opacity=1))
    
m
